In [1]:
import warnings
import pandas as pd
import torch
import random
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoModelForMaskedLM, AutoTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [2]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False  # 启用Cudnn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
warnings.filterwarnings('ignore')

# 创建自定义数据集类
class CitationDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, block_size=256):
        self.examples = []
        for text in texts:
            tokenized_text = tokenizer(text, truncation=True, padding='max_length', max_length=block_size, return_tensors='pt')
            self.examples.append(tokenized_text['input_ids'].squeeze())

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

df_unlabeled = pd.read_csv('./data/citing_paper_contexts_unlabeled.csv')
print(df_unlabeled.shape)
df_unlabeled.head()

(42091, 3)


,text,label_score,label
0,"Instead of mixing the representations, G-Mixup...",NaN,NaN
1,Graph Data Augmentation DropEdge [131] Randoml...,NaN,NaN
2,DropEdge [131] Randomly remove edges Perturbat...,NaN,NaN
3,"For G-Mixup, we use the same hyper-parameters ...",NaN,NaN
4,Few works have proposed methods to adapt mixup...,NaN,NaN


In [4]:
base_model_name = 'roberta-llama3.1405B-twitter-sentiment'
BASE_MODEL_PATH = f'./pretrain_models/{base_model_name}'
model = AutoModelForMaskedLM.from_pretrained(BASE_MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH)
tokenizer.save_pretrained(f'./citation_finetuned_models/{base_model_name}_itft')

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at ./pretrain_models/roberta-llama3.1405B-twitter-sentiment and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('./citation_finetuned_models/roberta-llama3.1405B-twitter-sentiment_itft\\tokenizer_config.json',
 './citation_finetuned_models/roberta-llama3.1405B-twitter-sentiment_itft\\special_tokens_map.json',
 './citation_finetuned_models/roberta-llama3.1405B-twitter-sentiment_itft\\vocab.json',
 './citation_finetuned_models/roberta-llama3.1405B-twitter-sentiment_itft\\merges.txt',
 './citation_finetuned_models/roberta-llama3.1405B-twitter-sentiment_itft\\added_tokens.json',
 './citation_finetuned_models/roberta-llama3.1405B-twitter-sentiment_itft\\tokenizer.json')

In [5]:
train_texts, valid_texts = train_test_split(df_unlabeled['text'].tolist(),test_size=0.2, random_state=seed)
train_dataset = CitationDataset(train_texts, tokenizer)
valid_dataset = CitationDataset(valid_texts, tokenizer)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
training_args = TrainingArguments(
    output_dir=f'./results/itft_{base_model_name}',  # select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=5,  # 轮数不要设置的太大
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    warmup_steps=10000,
    evaluation_strategy='steps',
    save_total_limit=2,
    eval_steps=250,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end=True,
    prediction_loss_only=True,
    report_to="none")

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)

trainer.train()
trainer.save_model(f'./citation_finetuned_models/{base_model_name}_itft')

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 